# Process old betting data (2009 ~ 2022)
Process old betting data (2-way money line).

Part of the 2023-2024 season data is incomplete.

Because the analysis rarely revisit pre-2009, I don't merge the data.

In [0]:
import pandas as pd
import numpy as np

# pull old data
df_old = pd.read_csv("/Volumes/nhl-databricks/data/betting/odds_past.csv", encoding="utf-7")

df_old.head(5)

Need to convert team tricode from the team name.

In [0]:
teamname = np.sort(df_old.Team.unique())
display(teamname)

In [0]:
# Given the limited number of the team names, I just manually write out tricode
match_tricode = [
    'ANA', 'ARI', 'ARI', 'ATL', 'BOS', 'BUF',
    'CAL', 'CAR', 'CHI', 'COL', 'CBJ', 'DAL',
    'DET', 'EDM', 'FLA', 'LAK', 'LAK', 
    'MIN', 'MTL', 'NYI', 'NYR', 'NYI', 'NYR', 'NSH', 'NJD', 'NJD',
    'OTT', 'PHI', 'ARI', 'PIT', 'SJS', 'SJS', 'SEA', 'SEA', 'SEA', 
    'STL', 'STL', 'TBL', 'TBL', 'TBL',
    'TOR', 'VAN', 'VGK', 'WSH', 'WPG', 'WPG' 
]

# Create matching dataframe
match_df = pd.DataFrame({'Team': teamname, 'tricode': match_tricode})
display(match_df)
# Merge the two dataframes
df_old = df_old.merge(match_df, on='Team', how='left')
display(df_old)
# Create a new

In [0]:
# If the Date column is length of three, need to add 1 to the year
df_old['year'] = np.where(
    df_old['Date'].astype(str).str.len() == 3, df_old['year'] + 1, df_old['year']
)
# Concatenate year and zero-padded Date (MMDD) columns
df_old['currentOddsDate'] = (
    df_old['year'].astype(str) +
    df_old['Date'].astype(str).str.zfill(4)
)
# Parse the combined string as a datetime, coercing errors to NaT
df_old['currentOddsDate'] = pd.to_datetime(
    df_old['currentOddsDate'],
    format='%Y%m%d',
    errors='coerce'
)
df_old.head(5)

In [0]:
# Convert Final to float
df_old['Final'] = pd.to_numeric(
    df_old['Final'],
    errors='coerce'
)
df_old['Final'] = df_old['Final'].fillna(0).astype(int)
display(df_old.tail(5))

In [0]:
# split the dataframe into two, odd rows and even rows, in order to get the home and away team
df_odd = df_old.iloc[0::2]
df_even = df_old.iloc[1::2]

df_odd = df_odd[['tricode', 'Final', 'Close']]
df_odd.columns = ['tricode_against', 'metric_score_against', 'away_odds_value']

df_even = df_even[['currentOddsDate', 'tricode', 'Final', 'Close']]
df_even.columns =  ['date', 'tricode_for', 'metric_score_for', 'home_odds_value']

In [0]:
df_odd = df_odd.reset_index(drop=True)
df_even = df_even.reset_index(drop=True)
df_odd_cleaned_old = pd.concat([df_odd, df_even], axis=1)
display(df_odd_cleaned_old)

Databricks visualization. Run in Databricks to view.

Join with actual game records

In [0]:
df_game = []
for iter_yr in range(2011, 2023):
    df_game.append(
        pd.read_csv(f'/Volumes/nhl-databricks/data/box/{iter_yr}_gamelist.csv')
    )
df_game = pd.concat(df_game, axis=0)
df_game = df_game.reset_index(drop=True)

# Convert 'date' column to datetime
df_game['date'] = pd.to_datetime(df_game['date'])

# Change metric score for and against to integer
df_game['metric_score_for'] = df_game['metric_score_for'].astype(int)
df_game['metric_score_against'] = df_game['metric_score_against'].astype(int)

display(df_game)

Databricks visualization. Run in Databricks to view.

In [0]:
# join with the odds data
df_merge = df_odd_cleaned_old.merge(
    df_game, 
    on=[
        'date',
        'tricode_for', 'tricode_against',
        'metric_score_for', 'metric_score_against'
    ], how='inner')
df_merge.display()

Databricks visualization. Run in Databricks to view.

In [0]:
df_legacy = df_merge[
    [
        'gameid', 'time_start', 'tricode_for', 'tricode_against', 
        'home_odds_value', 'away_odds_value', 'metric_score_for', 'metric_score_against',
        'period_ending', 'tricode_winteam'
    ]
]

# Add the latest data

In [0]:
df_2024 = pd.read_csv('/Volumes/nhl-databricks/data/betting/2024_odds.csv')
# column rename gameId to gameid
df_2024.rename(columns={'gameId': 'gameid'}, inplace=True)
# Currentoddsdate to datetime
df_2024['currentOddsDate'] = pd.to_datetime(df_2024['currentOddsDate'])
# only keep MONEY_LINE_2_WAY
df_2024 = df_2024[
    df_2024['odds_description'] == 'MONEY_LINE_2_WAY'
]

df_2024.display()

Databricks visualization. Run in Databricks to view.

In [0]:

# pull the gamelist data
df_game_2024 = pd.read_csv('/Volumes/nhl-databricks/data/box/2024_box.csv')

# merge the two dataframes
df_2024 = df_2024.merge(
    df_game_2024, on=['gameid'], how='inner'
)

df_2024.display()

In [0]:
df_new = df_2024[
    [
        'gameid', 'time_start', 'tricode_for', 'tricode_against', 
        'home_odds_value', 'away_odds_value', 'metric_score_for', 'metric_score_against',
        'period_ending', 'tricode_winteam'
    ]
]

# Merge with the old data and create the table
Create table in the Unity Catalog


In [0]:
# Ensure 'date' columns are datetime in both DataFrames before concatenation
#df_legacy['date'] = pd.to_datetime(df_legacy['date'])
#df_new['date'] = pd.to_datetime(df_new['date'])

# Concatenate using pandas
df_package = pd.concat(
    [df_legacy, df_new],
    axis=0
).reset_index(drop=True)

df_package['time_start'] = pd.to_datetime(df_package['time_start'], errors='coerce')

df_package.display()

In [0]:
# Save as a table with column comments using SQL struct definition
spark.sql("""
CREATE OR REPLACE TABLE `nhl-databricks`.data.odds (
    gameid DOUBLE COMMENT 'Unique game identifier',
    time_start TIMESTAMP COMMENT 'Game start time (UTC)',
    tricode_for STRING COMMENT 'Tricode for the home team',
    tricode_against STRING COMMENT 'Tricode for the away/opposing team',
    home_odds_value DOUBLE COMMENT 'Home team money line odds (2-way)',
    away_odds_value DOUBLE COMMENT 'Away team money line odds (2-way)',
    metric_score_for INT COMMENT 'Final score for the home team',
    metric_score_against INT COMMENT 'Final score for the away team',
    period_ending STRING COMMENT 'Game period when the game ended (REG, OT, SO)',
    tricode_winteam STRING COMMENT 'Tricode of the winning team'
)
""")

spark.createDataFrame(df_package).write.mode('overwrite').insertInto('`nhl-databricks`.data.odds')